
# Neural Network Classification Project

================================================================================

A comprehensive implementation of feed-forward neural networks for image classification
using TensorFlow/Keras on the MNIST dataset.

Author: AKAKPO Koffi Moïse

Date: August 2025

Purpose: Machine Learning Internship Application - Task 3

In [ ]:
# installation de tensorflow
!pip install tensorflow

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pickle
import warnings
warnings.filterwarnings('ignore')

import gradio as gr
import pickle
import cv2
from PIL import Image
import io
import base64
from datetime import datetime
import pandas as pd

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [3]:
class NeuralNetworkClassifier:
    """
    A comprehensive neural network classifier with hyperparameter tuning capabilities.
    """

    def __init__(self, input_shape, num_classes):
        """
        Initialize the neural network classifier.

        Args:
            input_shape (tuple): Shape of input data
            num_classes (int): Number of output classes
        """
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = None
        self.history = None
        self.best_model = None  # ✅ Attribut pour stocker le meilleur modèle
        self.best_params = None  # ✅ Attribut pour stocker les meilleurs paramètres
        self.best_accuracy = 0   # ✅ Attribut pour stocker la meilleure accuracy

    def create_model(self, hidden_layers=[128, 64], dropout_rate=0.2, learning_rate=0.001):
        """
        Create a feed-forward neural network architecture.

        Args:
            hidden_layers (list): List of hidden layer sizes
            dropout_rate (float): Dropout rate for regularization
            learning_rate (float): Learning rate for optimizer
        """
        model = keras.Sequential([
            layers.Flatten(input_shape=self.input_shape),
            layers.Dense(hidden_layers[0], activation='relu', name='hidden_1'),
            layers.Dropout(dropout_rate),
            layers.Dense(hidden_layers[1], activation='relu', name='hidden_2'),
            layers.Dropout(dropout_rate),
            layers.Dense(self.num_classes, activation='softmax', name='output')
        ])

        # Compile with appropriate optimizer and metrics
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(
            optimizer=optimizer,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        self.model = model
        return model

    def train(self, X_train, y_train, X_val, y_val, epochs=50, batch_size=32, verbose=1):
        """
        Train the neural network with early stopping and learning rate scheduling.

        Args:
            X_train, y_train: Training data
            X_val, y_val: Validation data
            epochs (int): Maximum number of epochs
            batch_size (int): Batch size for training
            verbose (int): Verbosity level
        """
        # Callbacks for better training
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=10,
                restore_best_weights=True,
                verbose=1
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-7,
                verbose=1
            )
        ]

        # Train the model
        self.history = self.model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks,
            verbose=verbose
        )

        return self.history

    def evaluate(self, X_test, y_test):
        """
        Evaluate the model on test data.

        Args:
            X_test, y_test: Test data

        Returns:
            dict: Evaluation metrics
        """
        # Get predictions
        y_pred_proba = self.model.predict(X_test, verbose=0)
        y_pred = np.argmax(y_pred_proba, axis=1)

        # Calculate metrics
        test_loss, test_accuracy = self.model.evaluate(X_test, y_test, verbose=0)

        # Classification report
        class_report = classification_report(y_test, y_pred, output_dict=True)

        # Confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)

        return {
            'test_loss': test_loss,
            'test_accuracy': test_accuracy,
            'predictions': y_pred,
            'probabilities': y_pred_proba,
            'classification_report': class_report,
            'confusion_matrix': conf_matrix
        }

    def hyperparameter_tuning(self, X_train, y_train, X_val, y_val, param_grid=None):
        """
        Perform hyperparameter tuning to find optimal parameters and save the best model.

        Args:
            X_train, y_train: Training data
            X_val, y_val: Validation data
            param_grid (dict): Dictionary of hyperparameters to test

        Returns:
            tuple: (best_params, results_df)
        """
        print("Starting hyperparameter tuning...")

        # Default parameter grid if none provided
        if param_grid is None:
            param_grid = {
                'learning_rate': [0.001, 0.01, 0.0001],
                'batch_size': [32, 64, 128],
                'hidden_layers': [[64, 32], [128, 64], [256, 128]]
            }

        best_accuracy = 0
        best_params = {}
        best_model = None
        results = []

        # Grid search
        for lr in param_grid['learning_rate']:
            for batch_size in param_grid['batch_size']:
                for hidden_layers in param_grid['hidden_layers']:
                    print(f"Testing: LR={lr}, Batch={batch_size}, Hidden={hidden_layers}")

                    # Create and train model
                    temp_model = self._create_temp_model(hidden_layers=hidden_layers, learning_rate=lr)

                    history = temp_model.fit(
                        X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=20,
                        batch_size=batch_size,
                        callbacks=[
                            keras.callbacks.EarlyStopping(
                                monitor='val_loss',
                                patience=5,
                                restore_best_weights=True,
                                verbose=0
                            )
                        ],
                        verbose=0
                    )

                    # Get best validation accuracy
                    val_accuracy = max(history.history['val_accuracy'])

                    results.append({
                        'learning_rate': lr,
                        'batch_size': batch_size,
                        'hidden_layers': str(hidden_layers),
                        'val_accuracy': val_accuracy
                    })

                    # ✅ Update best model if current is better
                    if val_accuracy > best_accuracy:
                        best_accuracy = val_accuracy
                        best_params = {
                            'learning_rate': lr,
                            'batch_size': batch_size,
                            'hidden_layers': hidden_layers
                        }
                        # Clone the best model
                        best_model = tf.keras.models.clone_model(temp_model)
                        best_model.set_weights(temp_model.get_weights())
                        best_model.compile(
                            optimizer=keras.optimizers.Adam(learning_rate=lr),
                            loss='sparse_categorical_crossentropy',
                            metrics=['accuracy']
                        )

        # ✅ Store best results in instance attributes
        self.best_model = best_model
        self.best_params = best_params
        self.best_accuracy = best_accuracy

        # Convert results to DataFrame for better visualization
        results_df = pd.DataFrame(results)
        print("\nHyperparameter Tuning Results:")
        print(results_df.sort_values('val_accuracy', ascending=False))

        print(f"\nBest Parameters: {best_params}")
        print(f"Best Validation Accuracy: {best_accuracy:.4f}")

        return best_params, results_df

    def _create_temp_model(self, hidden_layers=[128, 64], dropout_rate=0.2, learning_rate=0.001):
        """
        Create a temporary model for hyperparameter tuning.
        """
        model = keras.Sequential([
            layers.Flatten(input_shape=self.input_shape),
            layers.Dense(hidden_layers[0], activation='relu', name='hidden_1'),
            layers.Dropout(dropout_rate),
            layers.Dense(hidden_layers[1], activation='relu', name='hidden_2'),
            layers.Dropout(dropout_rate),
            layers.Dense(self.num_classes, activation='softmax', name='output')
        ])

        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(
            optimizer=optimizer,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        return model

    def save_best_model(self, filepath):
        """
        Save the best model to a file using pickle.

        Args:
            filepath (str): Path to save the model
        """
        if self.best_model is None:
            raise ValueError("No best model found. Run hyperparameter_tuning first.")

        model_data = {
            'model': self.best_model,
            'best_params': self.best_params,
            'best_accuracy': self.best_accuracy,
            'input_shape': self.input_shape,
            'num_classes': self.num_classes
        }

        with open(filepath, 'wb') as f:
            pickle.dump(model_data, f)

        print(f"✅ Best model saved to: {filepath}")
        print(f"   - Accuracy: {self.best_accuracy:.4f}")
        print(f"   - Parameters: {self.best_params}")

    @classmethod
    def load_best_model(cls, filepath):
        """
        Load a saved best model.

        Args:
            filepath (str): Path to the saved model

        Returns:
            NeuralNetworkClassifier: Loaded classifier instance
        """
        with open(filepath, 'rb') as f:
            model_data = pickle.load(f)

        # Create new instance
        classifier = cls(model_data['input_shape'], model_data['num_classes'])
        classifier.best_model = model_data['model']
        classifier.best_params = model_data['best_params']
        classifier.best_accuracy = model_data['best_accuracy']

        print(f"✅ Best model loaded from: {filepath}")
        print(f"   - Accuracy: {classifier.best_accuracy:.4f}")
        print(f"   - Parameters: {classifier.best_params}")

        return classifier

    def predict_with_best_model(self, X):
        """
        Make predictions using the best model.

        Args:
            X: Input data

        Returns:
            dict: Predictions and probabilities
        """
        if self.best_model is None:
            raise ValueError("No best model found. Run hyperparameter_tuning first.")

        probabilities = self.best_model.predict(X, verbose=0)
        predictions = np.argmax(probabilities, axis=1)

        return {
            'predictions': predictions,
            'probabilities': probabilities
        }

    def get_model_summary(self):
        """
        Get a summary of the best model and parameters.

        Returns:
            dict: Summary information
        """
        if self.best_model is None:
            return {"status": "No best model found. Run hyperparameter_tuning first."}

        return {
            'status': 'Best model available',
            'best_accuracy': self.best_accuracy,
            'best_params': self.best_params,
            'input_shape': self.input_shape,
            'num_classes': self.num_classes,
            'model_layers': [layer.name for layer in self.best_model.layers]
        }


In [4]:
def load_and_preprocess_data():
    """
    Load and preprocess the MNIST dataset.

    Returns:
        tuple: Preprocessed training and testing data
    """
    print("Loading MNIST dataset...")

    # Load data
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

    # Normalize pixel values to [0, 1]
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0

    # Split training data into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
    )

    print(f"Training set: {X_train.shape[0]} samples")
    print(f"Validation set: {X_val.shape[0]} samples")
    print(f"Test set: {X_test.shape[0]} samples")
    print(f"Image shape: {X_train.shape[1:]}")
    print(f"Number of classes: {len(np.unique(y_train))}")

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)


def main():
    """
    Exemple d'utilisation avec sauvegarde du meilleur modèle.
    """
    print("=" * 60)
    print("NEURAL NETWORK CLASSIFICATION PROJECT")
    print("=" * 60)

    # Set random seeds for reproducibility
    np.random.seed(42)
    tf.random.set_seed(42)

    # Load and preprocess data
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = load_and_preprocess_data()

    # Create classifier
    nn_classifier = NeuralNetworkClassifier(input_shape=(28, 28), num_classes=10)

    # Hyperparameter tuning (automatically saves best model)
    print("\n2. Hyperparameter Tuning")
    best_params, tuning_results = nn_classifier.hyperparameter_tuning(X_train, y_train, X_val, y_val)

    # ✅ Save the best model
    nn_classifier.save_best_model('best_mnist_model.pkl')

    # ✅ Test loading the model
    print("\n3. Testing Model Loading")
    loaded_classifier = NeuralNetworkClassifier.load_best_model('best_mnist_model.pkl')

    # ✅ Make predictions with best model
    print("\n4. Making Predictions with Best Model")
    results = loaded_classifier.predict_with_best_model(X_test[:100])

    # Evaluate on test set
    test_loss, test_accuracy = loaded_classifier.best_model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy with Best Model: {test_accuracy:.4f}")

    # ✅ Display model summary
    print("\n5. Model Summary")
    summary = loaded_classifier.get_model_summary()
    for key, value in summary.items():
        print(f"{key}: {value}")

    print("\n" + "=" * 60)
    print("✅ BEST MODEL SUCCESSFULLY SAVED AND LOADED!")
    print("=" * 60)

In [5]:
if __name__ == "__main__":
    main()

NEURAL NETWORK CLASSIFICATION PROJECT
Loading MNIST dataset...
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training set: 48000 samples
Validation set: 12000 samples
Test set: 10000 samples
Image shape: (28, 28)
Number of classes: 10

2. Hyperparameter Tuning
Starting hyperparameter tuning...
Testing: LR=0.001, Batch=32, Hidden=[64, 32]
Testing: LR=0.001, Batch=32, Hidden=[128, 64]
Testing: LR=0.001, Batch=32, Hidden=[256, 128]
Testing: LR=0.001, Batch=64, Hidden=[64, 32]
Testing: LR=0.001, Batch=64, Hidden=[128, 64]
Testing: LR=0.001, Batch=64, Hidden=[256, 128]
Testing: LR=0.001, Batch=128, Hidden=[64, 32]
Testing: LR=0.001, Batch=128, Hidden=[128, 64]
Testing: LR=0.001, Batch=128, Hidden=[256, 128]
Testing: LR=0.01, Batch=32, Hidden=[64, 32]
Testing: LR=0.01, Batch=32, Hidden=[128, 64]
Testing: LR=0.01, Batch=32, Hidden=[256, 128]
Testing: LR=0.01, Batch=64, Hidden=[64, 32]
Testing: LR=0.01, Batch=64, Hidden=[128, 64]
Testing: LR=0.01, Batch=64, Hidden=[256, 128]
Testing: LR=0

In [15]:
class MNISTPredictor:
    """
    Interface professionnelle pour la prédiction de chiffres manuscrits MNIST.
    """

    def __init__(self, model_path='best_mnist_model.pkl'):
        """
        Initialise le prédicteur avec le modèle sauvegardé.

        Args:
            model_path (str): Chemin vers le modèle sauvegardé
        """
        self.model = None
        self.model_info = None
        self.prediction_history = []

        # Charger le modèle
        self.load_model(model_path)

    def load_model(self, model_path):
        """
        Charge le modèle TensorFlow depuis le fichier pickle.

        Args:
            model_path (str): Chemin vers le modèle sauvegardé
        """
        try:
            with open(model_path, 'rb') as f:
                model_data = pickle.load(f)

            self.model = model_data['model']
            self.model_info = {
                'best_accuracy': model_data['best_accuracy'],
                'best_params': model_data['best_params'],
                'input_shape': model_data['input_shape'],
                'num_classes': model_data['num_classes']
            }

            print(f"✅ Modèle chargé avec succès!")
            print(f"   - Précision: {self.model_info['best_accuracy']:.4f}")
            print(f"   - Paramètres: {self.model_info['best_params']}")

        except FileNotFoundError:
            print(f"❌ Fichier modèle non trouvé: {model_path}")
            print("   Assurez-vous d'avoir exécuté l'entraînement d'abord.")
            self.model = None

        except Exception as e:
            print(f"❌ Erreur lors du chargement du modèle: {e}")
            self.model = None

    def preprocess_image(self, image):
        """
        Prétraite l'image pour la prédiction.

        Args:
            image: Image PIL ou array numpy

        Returns:
            np.array: Image prétraitée
        """
        # Convertir PIL en numpy si nécessaire
        if isinstance(image, Image.Image):
            image = np.array(image)

        # Convertir en niveaux de gris si couleur
        if len(image.shape) == 3:
            image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

        # Redimensionner à 28x28
        image = cv2.resize(image, (28, 28))

        # Inverser les couleurs (fond noir, écriture blanche)
        image = 255 - image

        # Normaliser entre 0 et 1
        image = image.astype('float32') / 255.0

        # Ajouter dimension batch
        image = np.expand_dims(image, axis=0)

        return image

    def predict(self, image):
        """
        Effectue la prédiction sur l'image.

        Args:
            image: Image à prédire

        Returns:
            tuple: (prédiction, probabilités, image_traitée, graphique)
        """
        if self.model is None:
            return "❌ Modèle non chargé", None, None, None

        try:
            # Prétraitement
            processed_image = self.preprocess_image(image)

            # Prédiction
            probabilities = self.model.predict(processed_image, verbose=0)[0]
            predicted_class = np.argmax(probabilities)
            confidence = probabilities[predicted_class]

            # Sauvegarder dans l'historique
            self.prediction_history.append({
                'timestamp': datetime.now().strftime("%H:%M:%S"),
                'prediction': int(predicted_class),
                'confidence': float(confidence),
                'probabilities': probabilities.tolist()
            })

            # Créer le graphique des probabilités
            prob_chart = self.create_probability_chart(probabilities)

            # Préparer l'image traitée pour affichage
            display_image = (processed_image[0] * 255).astype(np.uint8)

            # Résultat formaté
            result = f"🎯 **Prédiction: {predicted_class}**\n"
            result += f"🎲 **Confiance: {confidence:.2%}**\n\n"
            result += "📊 **Top 3 Probabilités:**\n"

            # Top 3 prédictions
            top_3_idx = np.argsort(probabilities)[-3:][::-1]
            for i, idx in enumerate(top_3_idx):
                result += f"{i+1}. Chiffre {idx}: {probabilities[idx]:.2%}\n"

            return result, probabilities, display_image, prob_chart

        except Exception as e:
            return f"❌ Erreur lors de la prédiction: {str(e)}", None, None, None

    def create_probability_chart(self, probabilities):
        """
        Crée un graphique des probabilités.

        Args:
            probabilities: Array des probabilités

        Returns:
            matplotlib.figure: Graphique
        """
        fig, ax = plt.subplots(figsize=(10, 6))

        classes = list(range(10))
        colors = plt.cm.viridis(np.linspace(0, 1, 10))

        bars = ax.bar(classes, probabilities, color=colors, alpha=0.8)

        # Mettre en évidence la prédiction
        max_idx = np.argmax(probabilities)
        bars[max_idx].set_color('#ff6b6b')
        bars[max_idx].set_alpha(1.0)

        ax.set_xlabel('Chiffres', fontsize=12, fontweight='bold')
        ax.set_ylabel('Probabilité', fontsize=12, fontweight='bold')
        ax.set_title('Distribution des Probabilités de Prédiction', fontsize=14, fontweight='bold')
        ax.set_ylim(0, 1)
        ax.grid(True, alpha=0.3)

        # Ajouter les valeurs sur les barres
        for i, (bar, prob) in enumerate(zip(bars, probabilities)):
            if prob > 0.01:  # Afficher seulement si > 1%
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                       f'{prob:.1%}', ha='center', va='bottom', fontsize=10)

        plt.tight_layout()
        return fig

    def get_model_info(self):
        """
        Retourne les informations du modèle.

        Returns:
            str: Informations formatées du modèle
        """
        if self.model is None:
            return "❌ Aucun modèle chargé"

        info = "🤖 **Informations du Modèle**\n\n"
        info += f"📈 **Précision de validation: {self.model_info['best_accuracy']:.4f}**\n"
        info += f"🔧 **Meilleurs paramètres:**\n"

        params = self.model_info['best_params']
        info += f"   • Taux d'apprentissage: {params['learning_rate']}\n"
        info += f"   • Taille de batch: {params['batch_size']}\n"
        info += f"   • Couches cachées: {params['hidden_layers']}\n\n"

        info += f"📊 **Architecture:**\n"
        info += f"   • Forme d'entrée: {self.model_info['input_shape']}\n"
        info += f"   • Nombre de classes: {self.model_info['num_classes']}\n"
        info += f"   • Nombre de paramètres: {self.model.count_params():,}\n"

        return info

    def get_prediction_history(self):
        """
        Retourne l'historique des prédictions sous forme de DataFrame.

        Returns:
            pd.DataFrame: Historique des prédictions
        """
        if not self.prediction_history:
            return pd.DataFrame(columns=['Heure', 'Prédiction', 'Confiance'])

        df = pd.DataFrame(self.prediction_history)
        df_display = pd.DataFrame({
            'Heure': df['timestamp'],
            'Prédiction': df['prediction'],
            'Confiance': df['confidence'].apply(lambda x: f"{x:.2%}")
        })

        return df_display.tail(10)  # Dernières 10 prédictions

    def clear_history(self):
        """
        Efface l'historique des prédictions.
        """
        self.prediction_history = []
        return pd.DataFrame(columns=['Heure', 'Prédiction', 'Confiance'])


In [16]:
def create_gradio_interface():
    """
    Crée l'interface Gradio professionnelle.

    Returns:
        gr.Interface: Interface Gradio configurée
    """

    # Initialiser le prédicteur
    predictor = MNISTPredictor()

    # Interface CSS personnalisé
    custom_css = """
    .gradio-container {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }

    .main-header {
        text-align: center;
        color: white;
        padding: 20px;
        margin-bottom: 20px;
        background: rgba(0,0,0,0.1);
        border-radius: 10px;
    }

    .prediction-box {
        background: rgba(255,255,255,0.9);
        border-radius: 10px;
        padding: 15px;
        margin: 10px 0;
    }
    """

    with gr.Blocks(css=custom_css, title="MNIST Digit Classifier") as interface:

        gr.HTML("""
        <div class="main-header">
            <h1>🔢 Classificateur de Chiffres Manuscrits MNIST</h1>
            <p>Interface professionnelle utilisant un réseau de neurones TensorFlow</p>
            <p><strong>Auteur:</strong> AKAKPO Koffi Moïse | <strong>Projet:</strong> Machine Learning Internship</p>
            <p><em>💡 Astuce: Vous pouvez télécharger des images de chiffres manuscrits ou utiliser des captures d'écran</em></p>
        </div>
        """)

        with gr.Tab("🎯 Prédiction"):
            with gr.Row():
                with gr.Column(scale=1):
                    # Zone de téléchargement d'image
                    input_image = gr.Image(
                        label="✏️ Téléchargez ou collez une image de chiffre (0-9)",
                        type="pil",
                        height=280,
                        width=280
                    )

                    # Boutons
                    with gr.Row():
                        predict_btn = gr.Button("🔍 Prédire", variant="primary", size="lg")
                        clear_btn = gr.Button("🗑️ Effacer", variant="secondary")

                with gr.Column(scale=1):
                    # Résultats
                    prediction_output = gr.Markdown(
                        label="📊 Résultat de la Prédiction",
                        value="👆 Téléchargez une image de chiffre et cliquez sur 'Prédire'"
                    )

                    # Image traitée
                    processed_image = gr.Image(
                        label="🖼️ Image Traitée (28x28)",
                        type="numpy"
                    )

            # Graphique des probabilités
            probability_chart = gr.Plot(label="📈 Distribution des Probabilités")

        with gr.Tab("📊 Historique"):
            with gr.Row():
                history_df = gr.Dataframe(
                    label="📜 Dernières Prédictions",
                    interactive=False
                )

            with gr.Row():
                refresh_history_btn = gr.Button("🔄 Actualiser", variant="secondary")
                clear_history_btn = gr.Button("🗑️ Effacer Historique", variant="secondary")

        with gr.Tab("ℹ️ Informations du Modèle"):
            model_info_output = gr.Markdown(
                value=predictor.get_model_info(),
                label="🤖 Détails du Modèle"
            )

        # Gestion des événements
        def predict_and_update(image):
            if image is None:
                return "⚠️ Veuillez télécharger une image d'abord!", None, None, None

            result, probs, processed_img, chart = predictor.predict(image)
            return result, processed_img, chart, predictor.get_prediction_history()

        def clear_canvas():
            return None, "👆 Téléchargez une image de chiffre et cliquez sur 'Prédire'", None, None

        def refresh_history():
            return predictor.get_prediction_history()

        def clear_pred_history():
            return predictor.clear_history()

        # Connexions des événements
        predict_btn.click(
            fn=predict_and_update,
            inputs=[input_image],
            outputs=[prediction_output, processed_image, probability_chart, history_df]
        )

        clear_btn.click(
            fn=clear_canvas,
            outputs=[input_image, prediction_output, processed_image, probability_chart]
        )

        refresh_history_btn.click(
            fn=refresh_history,
            outputs=[history_df]
        )

        clear_history_btn.click(
            fn=clear_pred_history,
            outputs=[history_df]
        )

    return interface

def main():
    """
    Lance l'application Gradio.
    """
    print("🚀 Lancement de l'interface MNIST Classifier...")

    # Créer et lancer l'interface
    interface = create_gradio_interface()

    # Lancer l'application
    interface.launch(
        share=True,          # Créer un lien public
        server_name="0.0.0.0",  # Accessible depuis toutes les IPs
        server_port=7860,    # Port par défaut
        show_error=True,     # Afficher les erreurs
        quiet=False          # Mode verbose
    )


In [17]:
if __name__ == "__main__":
    main()

🚀 Lancement de l'interface MNIST Classifier...
✅ Modèle chargé avec succès!
   - Précision: 0.9805
   - Paramètres: {'learning_rate': 0.001, 'batch_size': 128, 'hidden_layers': [256, 128]}
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3dc38f5646da82b337.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
